## 심화 학습 및 최신 기술 동향

### 내용
1. 심화 학습 주제 소개
    - 딥러닝의 심화 학습 주제 소개
        - 강화 학습(Reinforcement Learning)
        - 생성적 적대 신경망(GANs, Generative Adversarial Networks)
        - Transformer 및 BERT, GPT 등 최신 언어 모델
        - 메타 학습(Meta-Learning) 및 Few-Shot Learning

2. 강화 학습(Reinforcement Learning)
    - 에이전트와 환경, 보상 및 정책
    - 주요 알고리즘: Q-Learning, Deep Q-Networks(DQN), Policy Gradients

- DQN 구현 예제

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np

# 환경 설정
env = gym.make('CartPole-v1')

# 신경망 모델 정의
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# DQN 에이전트 정의
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        act_values = self.model(state)
        return torch.argmax(act_values, dim=1).item()

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * torch.max(self.model(torch.FloatTensor(next_state).unsqueeze(0))).item())
            target_f = self.model(torch.FloatTensor(state).unsqueeze(0))
            target_f[0][action] = target
            self.optimizer.zero_grad()
            loss = self.criterion(target_f, torch.FloatTensor(state).unsqueeze(0))
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# DQN 학습
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
episodes = 1000
batch_size = 32

for e in range(episodes):
    state = env.reset()
    for time in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"Episode {e+1}/{episodes}, score: {time}")
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

3. 생성적 적대 신경망(GANs)
    - GAN의 기본 개념
        - 생성자(Generator)와 판별자(Discriminator)의 경쟁적 학습
    - 간단한 GAN 구현 예제

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 데이터셋 로드 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 생성자 모델 정의
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)
    
# 판별자 모델 정의
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)
    

# 모델, 손실 함수, 옵티마이저 설정
generator = Generator()
discriminator = Discriminator()
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002)

# GAN 학습
num_epochs = 50
for epoch in range(num_epochs):
    for real_images, _ in train_loader:
        # 진짜 이미지에 대한 판별자 학습
        real_images = real_images.view(real_images.size(0), -1)
        real_labels = torch.ones(real_images.size(0), 1)
        fake_labels = torch.zeros(real_images.size(0), 1)
        
        outputs = discriminator(real_images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        # 가짜 이미지에 대한 판별자 학습
        z = torch.randn(real_images.size(0), 100)
        fake_images = generator(z)
        outputs = discriminator(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        # 판별자 최종 손실 및 역전파
        d_loss = d_loss_real + d_loss_fake
        optimizer_d.zero_grad()
        d_loss.backward()
        optimizer_d.step()

        # 생성자 학습
        z = torch.randn(real_images.size(0), 100)
        fake_images = generator(z)
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)

        optimizer_g.zero_grad()
        g_loss.backward()
        optimizer_g.step()

    print(f"Epoch [{epoch}/{num_epochs}], d_loss: {d_loss.item()}, g_loss: {g_loss.item()}, D(x): {real_score.mean().item()}, D(G(z)): {fake_score.mean().item()}")

4. Transformer 및 최신 언어 모델

    - Transformer 개념 및 구조
        - Self-Attention 메커니즘
        - Encoder-Decoder 아키텍처
    - BERT, GPT 등의 최신 언어 모델 개요 및 활용

In [ ]:
from transformers import BertTokenizer, BertModel

# BERT 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# 텍스트 입력
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors="pt")

# BERT 모델 예측
outputs = model(**inputs)

5. 메타 학습(Meta-Learning) 및 Few-Shot Learning

    - 메타 학습 개념
        - 모델이 새로운 태스크를 빠르게 학습할 수 있도록 하는 학습 방법
    - Few-Shot Learning 개념 및 활용

In [ ]:
from torchmeta.datasets.helpers import miniimagenet
from torchmeta.utils.data import BatchMetaDataLoader

# Mini-ImageNet 데이터셋 로드
dataset = miniimagenet("data", ways=5, shots=1, test_shots=15, meta_train=True, download=True)
dataloader = BatchMetaDataLoader(dataset, batch_size=16, num_workers=4)

for batch in dataloader:
    print(batch["train"][0].shape)  # (batch_size, ways, shots, channels, height, width)
    print(batch["test"][0].shape)   # (batch_size, ways, test_shots, channels, height, width)